In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import xgboost as xgb
%matplotlib inline

## Data preprocesing

In [2]:
m = pd.read_csv('./hltv_parser/matches.csv')
IEM_matches = m[m['tourney'] == 4901]
before_IEM = m[m['tourney'] != 4901]

In [3]:
from feature_engineering import process_data

x, y, lineups_map_info, lineups_elo = process_data(before_IEM)

/Users/valery_khudziev/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/valery_khudziev/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/Users/valery_khudziev/Documents/cs-go-match-winner/feature_engineering.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .

## Bayesian bernoulli

In [3]:
from bayesian_bernoulli import BayesianPredictor

clf = BayesianPredictor()

/Users/valery_khudziev/anaconda3/lib/python3.7/site-packages/distributed/utils.py:133: RuntimeWarning: Couldn't detect a suitable IP address for reaching '8.8.8.8', defaulting to '127.0.0.1': [Errno 51] Network is unreachable
  RuntimeWarning,


In [4]:
before_IEM_target = before_IEM.apply(lambda d: 1 if d['t1_score'] > d['t2_score'] else 0, axis=1)

In [ ]:
clf.fit(before_IEM, before_IEM_target)
with open('bayesian_4901.pkl', 'wb') as file:
    pickle.dump(clf, file)

## XGBoost

In [349]:
params_xgb = {'max_depth':29,
              'objective':'binary:logistic',
              'min_child_weight': 1,
              'eta'      :0.1,
              'subsample':0.8,
              'lambda '  :4,
              'n_estimators':4000,
              'colsample_bytree ':1,
              'colsample_bylevel':1
              }

In [4]:
tmp = x

In [257]:
x.shape

(20364, 29)

In [14]:
x = tmp

In [15]:
y = before_IEM.apply(lambda d: 1 if d['t1_score'] > d['t2_score'] else 0, axis=1).reset_index().drop(columns=['index'])

In [16]:
before_IEM = before_IEM.reset_index()

In [17]:
idxs = (before_IEM['map'] != 'Cache') & (before_IEM['map'] != 'Cobblestone') & (before_IEM['map'] != 'Mirage_ce')\
    & (before_IEM['map'] != 'Inferno_se') & (before_IEM['map'] != 'Train_se')\
    & (before_IEM['map'] != 'Nuke_se') & (before_IEM['map'] != 'Season')

In [18]:
x = x[idxs]
y = y[idxs]

In [21]:
def keep_map_stat(tr, maps):
    x = tr
    x['map'] = maps.reset_index().drop(columns=['index'])
    x['t1_win_percent'] = np.zeros(x.shape[0])
    x['t1_played_percent'] = np.zeros(x.shape[0])
    x['t2_win_percent'] = np.zeros(x.shape[0])
    x['t2_played_percent'] = np.zeros(x.shape[0])
    for idx in x.index:
        x.at[idx, 't1_win_percent'] = x['t1_' + x.loc[idx]['map'] + '_win_percent'].loc[idx]
        x.at[idx, 't1_played_percent'] = x['t1_' + x.loc[idx]['map'] + '_played_percent'].loc[idx]
        x.at[idx, 't2_win_percent'] = x['t2_' + x.loc[idx]['map'] + '_win_percent'].loc[idx]
        x.at[idx, 't2_played_percent'] = x['t2_' + x.loc[idx]['map'] + '_played_percent'].loc[idx]
    x = x[['t1_win_percent', 't1_played_percent', 't2_win_percent',
           't2_played_percent', 'elo_diff', 't1_elo', 't2_elo']]
    return x

In [352]:
train_set = xgb.DMatrix(x, y)
clf = xgb.train(params_xgb, train_set)

In [20]:
import itertools

def make_x_pred(clf, x_pred, lineups_map_info, lineups_elo):
    cs_maps = ('Dust2', 'Inferno', 'Mirage', 'Nuke', 'Overpass', 'Train', 'Vertigo')
    stats = ('times_played', 'wins')
    x = pd.DataFrame()
    for cs_map, stat in itertools.product(cs_maps, stats):
        x['t1_' + cs_map + '_' + stat] = np.zeros(x_pred.shape[0])
        x['t2_' + cs_map + '_' + stat] = np.zeros(x_pred.shape[0])
    for i in range(x_pred.shape[0]):
        for cs_map, stat in itertools.product(cs_maps, stats):
            x.at[i, 't1_' + cs_map + '_' + stat] = lineups_map_info[x_pred.iloc[i]['t1_lineup']][cs_map + stat]
            x.at[i, 't2_' + cs_map + '_' + stat] = lineups_map_info[x_pred.iloc[i]['t2_lineup']][cs_map + stat]
    for cs_map in cs_maps:
        x['t1_' + cs_map + '_played_percent'] = np.zeros(x.shape[0])
        x['t2_' + cs_map + '_played_percent'] = np.zeros(x.shape[0])
        x['t1_' + cs_map + '_win_percent'] = np.zeros(x.shape[0])
        x['t2_' + cs_map + '_win_percent'] = np.zeros(x.shape[0])
    
    x['elo_diff'] = np.zeros(x_pred.shape[0])
    x['t1_elo'] = np.zeros(x_pred.shape[0])
    x['t2_elo'] = np.zeros(x_pred.shape[0])
    for idx in x.index:
        s1 = 0
        s2 = 0
        for cs_map in cs_maps:
            s1 += x.loc[idx]['t1_' + cs_map + '_times_played']
            s2 += x.loc[idx]['t2_' + cs_map + '_times_played']
        for cs_map in cs_maps:
            t1_times_played = x.loc[idx]['t1_' + cs_map + '_times_played']
            t2_times_played = x.loc[idx]['t2_' + cs_map + '_times_played']
            t1_wins = x.loc[idx]['t1_' + cs_map + '_wins']
            t2_wins = x.loc[idx]['t2_' + cs_map + '_wins']
            x.at[idx, 't1_' + cs_map + '_played_percent'] = t1_times_played / s1 if s1 != 0 else 0
            x.at[idx, 't1_' + cs_map + '_win_percent'] = t1_wins / t1_times_played if t1_times_played != 0 else 0
            x.at[idx, 't2_' + cs_map + '_played_percent'] = t2_times_played / s2 if s2 != 0 else 0
            x.at[idx, 't2_' + cs_map + '_win_percent'] = t2_wins / t2_times_played if t2_times_played != 0 else 0
        x.at[idx, 'elo_diff'] = lineups_elo[x_pred.iloc[idx]['t1_lineup']] - lineups_elo[x_pred.iloc[idx]['t2_lineup']]
        x.at[idx, 't1_elo'] = lineups_elo[x_pred.iloc[idx]['t1_lineup']]
        x.at[idx, 't2_elo'] = lineups_elo[x_pred.iloc[idx]['t2_lineup']]
    to_drop = ['t1_' + cs_map + '_' + stat for cs_map, stat in itertools.product(cs_maps, stats)] +\
        ['t2_' + cs_map + '_' + stat for cs_map, stat in itertools.product(cs_maps, stats)]
    x = x.drop(columns=to_drop)
    return x

In [354]:
x_pred = make_x_pred(clf, IEM_matches, lineups_map_info, lineups_elo)

In [343]:
y_test = IEM_matches.apply(lambda d: 1 if d['t1_score'] > d['t2_score'] else 0, axis=1)

In [ ]:
test_set = xgb.DMatrix(x_pred)
IEM_pred = clf.predict(test_set)

In [355]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_test, IEM_pred)

0.701923076923077

## CatBoost

In [356]:
x = tmp

In [357]:
y = before_IEM.apply(lambda d: 1 if d['t1_score'] > d['t2_score'] else 0, axis=1).reset_index().drop(columns=['index'])

In [358]:
idxs = (before_IEM['map'] != 'Cache') & (before_IEM['map'] != 'Cobblestone') & (before_IEM['map'] != 'Mirage_ce')\
    & (before_IEM['map'] != 'Inferno_se') & (before_IEM['map'] != 'Train_se')\
    & (before_IEM['map'] != 'Nuke_se') & (before_IEM['map'] != 'Season')

In [359]:
x = x[idxs]
y = y[idxs]

In [22]:
x = keep_map_stat(x, before_IEM['map'])

/Users/valery_khudziev/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/valery_khudziev/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/valery_khudziev/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to 

In [23]:
from catboost import CatBoostClassifier

In [33]:
clf = CatBoostClassifier(iterations=500,
                        learning_rate=0.1,
                        depth=15,
                        l2_leaf_reg=4,
                        loss_function='Logloss',
                        )

In [ ]:
clf.fit(x, y)

0:	learn: 0.6821544	total: 3.26s	remaining: 27m 5s
1:	learn: 0.6713019	total: 7.18s	remaining: 29m 47s
2:	learn: 0.6622381	total: 10.8s	remaining: 29m 52s
3:	learn: 0.6543562	total: 14.8s	remaining: 30m 36s
4:	learn: 0.6465598	total: 22.6s	remaining: 37m 19s
5:	learn: 0.6398712	total: 25.6s	remaining: 35m 5s
6:	learn: 0.6334996	total: 28.4s	remaining: 33m 23s
7:	learn: 0.6286831	total: 31.2s	remaining: 31m 58s
8:	learn: 0.6227236	total: 33.9s	remaining: 30m 50s
9:	learn: 0.6179217	total: 36.6s	remaining: 29m 55s
10:	learn: 0.6133928	total: 39.3s	remaining: 29m 5s
11:	learn: 0.6091392	total: 44.9s	remaining: 30m 26s
12:	learn: 0.6054732	total: 48.4s	remaining: 30m 14s
13:	learn: 0.6018832	total: 52.4s	remaining: 30m 17s
14:	learn: 0.5977166	total: 56.2s	remaining: 30m 18s
15:	learn: 0.5940192	total: 59.5s	remaining: 30m
16:	learn: 0.5912162	total: 1m 2s	remaining: 29m 44s
17:	learn: 0.5884763	total: 1m 5s	remaining: 29m 20s
18:	learn: 0.5856726	total: 1m 8s	remaining: 29m
19:	learn: 0.5

153:	learn: 0.4096135	total: 8m 47s	remaining: 19m 46s
154:	learn: 0.4088576	total: 8m 51s	remaining: 19m 42s
155:	learn: 0.4079369	total: 8m 54s	remaining: 19m 38s
156:	learn: 0.4068173	total: 8m 58s	remaining: 19m 36s
157:	learn: 0.4058774	total: 9m 3s	remaining: 19m 36s
158:	learn: 0.4047492	total: 9m 6s	remaining: 19m 33s
159:	learn: 0.4040671	total: 9m 9s	remaining: 19m 28s
160:	learn: 0.4033831	total: 9m 12s	remaining: 19m 24s
161:	learn: 0.4024395	total: 9m 17s	remaining: 19m 23s
162:	learn: 0.4017190	total: 9m 21s	remaining: 19m 20s
163:	learn: 0.4009882	total: 9m 25s	remaining: 19m 18s
164:	learn: 0.4002113	total: 9m 28s	remaining: 19m 14s
165:	learn: 0.3993692	total: 9m 32s	remaining: 19m 11s
166:	learn: 0.3987329	total: 9m 35s	remaining: 19m 8s
167:	learn: 0.3982343	total: 9m 39s	remaining: 19m 4s
168:	learn: 0.3975074	total: 9m 42s	remaining: 18m 59s
169:	learn: 0.3969138	total: 9m 45s	remaining: 18m 56s
170:	learn: 0.3962452	total: 9m 48s	remaining: 18m 52s
171:	learn: 0.3

In [27]:
x_pred = make_x_pred(clf, IEM_matches, lineups_map_info, lineups_elo)

In [28]:
x_pred = keep_map_stat(x_pred, IEM_matches['map'])

In [29]:
IEM_pred = clf.predict(x_pred)

In [31]:
y_test = IEM_matches.apply(lambda d: 1 if d['t1_score'] > d['t2_score'] else 0, axis=1)

In [32]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_test, IEM_pred)

0.596153846153846